<a href="https://colab.research.google.com/github/yogasgm/prototype_finetuning_pytorch/blob/main/Prototype_Multiclass_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Importing libraries

In [ ]:
!pip install transformers

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import random
import shutil
import sys
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
df = pd.read_excel('/content/labeled_data_20250426_171037.xlsx')

# Assuming 'df' from previous cells is your data:
train_df = df.copy()  # Create a copy of 'df' and name it 'train_df'

In [ ]:
# Assuming your CSV file is named 'All Data_labeled.csv'
file_path = '/content/labeled_data_20250426_171037.xlsx'  # Assign the path to a variable
train_df = pd.read_excel(file_path) # Use the variable in pd.read_csv

# Setting seed for reproducibility

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
set_seed(43)

# Downloading dataset

In [ ]:
# Load the dataset directly from the Colab file system
file_path = '/content/labeled_data_20250426_171037.xlsx'  # Adjust the filename as needed
train_df = pd.read_excel(file_path)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16849 entries, 0 to 16848
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             16849 non-null  object
 1   Predicted_Label  16849 non-null  object
dtypes: object(2)
memory usage: 263.4+ KB


In [ ]:
possible_labels = train_df.Predicted_Label.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Individuals': 0, 'Uncategorized': 1, 'Governance': 2, 'Corporations': 3}

In [ ]:
train_df['Predicted_Label'] = train_df.Predicted_Label.replace(label_dict)

<ipython-input-10-13e07c457203>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['Predicted_Label'] = train_df.Predicted_Label.replace(label_dict)


In [ ]:
train_df.head()

,text,Predicted_Label
0,",i would love to see s britghtmark recycling i...",0
1,",this is very refreshing to see even though i ...",0
2,",""that citizen bread to beer to bread is it re...",0
3,cause it first theyre taking wheat bread waste...,1
4,then they use this beer waste to make a second...,1


# Preparing the tokenizer

In [ ]:
#Set Max Lenght, maksimal 512 (BERT)
MAX_LEN = 512

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
#download the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.text = df['text']
        self.targets = self.df['Predicted_Label']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)
        }

# Splitting & Tokenizing Dataset

In [ ]:
# Adjusting the train/validation/test split
train_df, temp_df = train_test_split(train_df, test_size=0.2, random_state=43)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=43)

# Reset the indices
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
# For training set
print(train_df['Predicted_Label'].value_counts())

# For validation set
print(val_df['Predicted_Label'].value_counts())

# For validation set
print(test_df['Predicted_Label'].value_counts())

Predicted_Label
0    6134
1    3216
2    2646
3    1483
Name: count, dtype: int64
Predicted_Label
0    773
1    418
2    323
3    171
Name: count, dtype: int64
Predicted_Label
0    748
1    414
2    344
3    179
Name: count, dtype: int64


In [ ]:
train_df.shape

(13479, 2)

In [ ]:
val_df.shape

(1685, 2)

In [ ]:
val_df

,text,Predicted_Label
0,as soon as the bacteria make their way into a ...,1
1,",what happened to the trash they vote for demo...",2
2,",why is the trash going to other states pollut...",2
3,",""world",1
4,",speaking of tradeoffs why not discuss the dam...",1
...,...,...
1680,",you can also own stuff in a circular economy ...",0
1681,",i might lose my sleep when i seriously start ...",0
1682,",when a zombie apocalypse occur i know who to ...",0
1683,",they need to build about more trash to power...",2


In [ ]:
# Create the CustomDataset for each set
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN)

In [ ]:
len(train_dataset)

13479

# Setting hyperparameters

In [ ]:
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 3e-5

In [ ]:
# Data loaders
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE)
val_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE)

In [ ]:
# Checking for available device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

# Additional functions for loading and saving checkpoints

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

# Training the Model

Defining and Initializing the BERT Classification Model

In [ ]:
# Define the model
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, len(label_dict))  # Adjust the final layer to match the number of classes

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

In [ ]:
# Initialize the model
model = BERTClass()
model.to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

Setting Up the Loss Function and Optimizer

In [ ]:
# Define loss function (CrossEntropyLoss for multi-class classification)
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)

Initialization of Validation Target and Output Lists

In [ ]:
val_targets=[]
val_outputs=[]

Training and Validation Loop with Early Stopping

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, optimizer, checkpoint_path, best_model_path, patience):
  valid_loss_min = np.inf
  no_improve = 0

  for epoch in range(1, n_epochs+1):
    train_loss = 0.0
    valid_loss = 0.0
    model.train()
    print('############# Epoch {}: Training Start #############'.format(epoch))

    for batch_idx, data in enumerate(training_loader):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()

        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))

    print('############# Epoch {}: Training End #############'.format(epoch))

    print('############# Epoch {}: Validation Start #############'.format(epoch))
    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)

            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))

            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    print('############# Epoch {}: Validation End #############'.format(epoch))

    train_loss = train_loss/len(training_loader)
    valid_loss = valid_loss/len(validation_loader)

    print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
    epoch, train_loss, valid_loss))

    checkpoint = {
      'epoch': epoch + 1,
      'valid_loss_min': valid_loss,
      'state_dict': model.state_dict(),
      'optimizer': optimizer.state_dict()
    }

    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
        torch.save(checkpoint, best_model_path)
        valid_loss_min = valid_loss
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= patience:
          print("Early stopping due to no improvement in validation loss")
          break

  return model

In [ ]:
# Save checkpoint

ckpt_path = '/content/ckpt.pth'
best_model_path = '/content/best_model.pth'

# Start Train

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path, patience = 2)

############# Epoch 1: Training Start #############
############# Epoch 1: Training End #############
############# Epoch 1: Validation Start #############
############# Epoch 1: Validation End #############
Epoch: 1 	Avgerage Training Loss: 0.000888 	Average Validation Loss: 0.005512
Validation loss decreased (inf --> 0.005512).  Saving model ...
############# Epoch 2: Training Start #############
############# Epoch 2: Training End #############
############# Epoch 2: Validation Start #############
############# Epoch 2: Validation End #############
Epoch: 2 	Avgerage Training Loss: 0.000431 	Average Validation Loss: 0.005325
Validation loss decreased (0.005512 --> 0.005325).  Saving model ...
############# Epoch 3: Training Start #############
############# Epoch 3: Training End #############
############# Epoch 3: Validation Start #############
############# Epoch 3: Validation End #############
Epoch: 3 	Avgerage Training Loss: 0.000203 	Average Validation Loss: 0.006988
#########

In [ ]:
# Load the saved checkpoint
model, optimizer, start_epoch, valid_loss_min = load_ckp(best_model_path, model, optimizer)

print(f'The validation loss of the best saved model is: {valid_loss_min}')

The validation loss of the best saved model is: 0.004918792156052359


# Test

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# switch model out of training mode
model.eval()

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
predictions = []
true_labels = []

In [ ]:
# Data loaders
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE)

In [ ]:
# iterate over test data
for data in test_data_loader:
    # move tensors to GPU if available
    ids = data['input_ids'].to(device, dtype = torch.long)
    mask = data['attention_mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.long)

    # compute predicted outputs
    outputs = model(ids, mask, token_type_ids)

    # convert output probabilities to predicted class
    _, pred = torch.max(outputs, dim=1)

    predictions.extend(pred.cpu().detach().numpy().tolist())
    true_labels.extend(targets.cpu().detach().numpy().tolist())

In [ ]:
# calculate and print classification report
print(classification_report(true_labels, predictions, target_names=label_dict.keys()))

               precision    recall  f1-score   support

  Individuals       0.91      0.86      0.88       748
Uncategorized       0.80      0.71      0.75       414
   Governance       0.76      0.89      0.82       344
 Corporations       0.63      0.73      0.67       179

     accuracy                           0.81      1685
    macro avg       0.77      0.80      0.78      1685
 weighted avg       0.82      0.81      0.82      1685



In [ ]:
# print confusion matrix
print(confusion_matrix(true_labels, predictions))

[[645  40  34  29]
 [ 33 293  50  38]
 [ 12  17 305  10]
 [ 21  18  10 130]]


# Predict New Text Input

In [ ]:
def predict_target(model, tokenizer, text, max_len=128):
    model.eval()

    # Preprocess the text
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    ids = inputs['input_ids'].to(device)
    mask = inputs['attention_mask'].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)

    outputs = model(ids, mask, token_type_ids)
    outputs = torch.softmax(outputs, dim=1).detach().cpu().numpy()

    # Get the label with highest probability
    predicted_label_idx = np.argmax(outputs, axis=1)

    # Reverse the label dictionary to get the name of the class
    reverse_label_dict = {v: k for k, v in label_dict.items()}
    predicted_label = reverse_label_dict[predicted_label_idx[0]]

    # Get probabilities for each class
    probabilities = outputs[0]

    return predicted_label, probabilities


In [ ]:
text = ""
predicted_label, probabilities = predict_target(model, tokenizer, text)
print(f"The predicted label for the text is: {predicted_label}")
print(f"The probabilities for each class are: {probabilities}")

The predicted label for the text is: Individuals
The probabilities for each class are: [0.9267561  0.06618681 0.00391957 0.00313754]


In [ ]:
text = ""
predicted_label, probabilities = predict_target(model, tokenizer, text)
print(f"The predicted label for the text is: {predicted_label}")
print(f"The probabilities for each class are: {probabilities}")

The predicted label for the text is: Individuals
The probabilities for each class are: [0.9267561  0.06618681 0.00391957 0.00313754]


In [ ]:
text = ""
predicted_label, probabilities = predict_target(model, tokenizer, text)
print(f"The predicted label for the text is: {predicted_label}")
print(f"The probabilities for each class are: {probabilities}")

The predicted label for the text is: Individuals
The probabilities for each class are: [0.9267561  0.06618681 0.00391957 0.00313754]


# New Prediciton

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
# 1. Load tokenizer dan model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # atau nama model pre-trained

# Instead of using from_pretrained, load the state dictionary directly
model = BERTClass() # Assuming BERTClass is your model definition
model.load_state_dict(torch.load('/content/best_model.pth')['state_dict']) # Load the state dictionary from your saved checkpoint
model.to(device) # If using GPU
model.eval()  # set model ke eval mode

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
# 2. Load dataset baru
new_data = pd.read_excel('/content/2_data_unlabeled_prediction.xlsx')  # atau .csv
texts = new_data['text'].tolist()  # ganti 'text' sesuai nama kolommu

In [ ]:
# 3. Preprocessing: tokenisasi
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

In [ ]:
# 4. Predict in Batches
batch_size = 8  # Choose an appropriate batch size based on your GPU memory

all_predictions = []
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

    with torch.no_grad():
        # Access the inputs individually
        outputs = model(inputs['input_ids'].to(device), inputs['attention_mask'].to(device), inputs['token_type_ids'].to(device))
        logits = outputs  # outputs is already the logits
        batch_predictions = torch.argmax(logits, dim=1).cpu().numpy()
        all_predictions.extend(batch_predictions)

predictions = np.array(all_predictions)

In [ ]:
# 5. Simpan hasil prediksi
new_data['prediction'] = predictions
new_data.to_excel('prediction_result.xlsx', index=False)